In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../creme/')
import creme
import utils
import custom_model
import pandas as pd
import matplotlib.pyplot as plt

2024-06-06 03:13:11.921360: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Load Enformer and example sequences

In [2]:
track_index = [5111]
model = custom_model.Enformer(track_index=track_index)
target_df = pd.read_csv(f'../data/enformer_targets_human.txt', sep='\t')
cell_line_info = {i: [t, utils.clean_cell_name(target_df.iloc[t]['description'])] for i, t in
              enumerate(track_index)}

2024-06-06 03:13:13.464975: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-06 03:13:15.355969: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 740 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:85:00.0, compute capability: 8.0
2024-06-06 03:13:27.147872: W tensorflow/tsl/framework/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 6.0KiB (rounded to 6144)requested by op Identity
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run Identity: Dst tensor is not initialized. [Op:Identity]

In [ ]:
fasta_path = f'../data/GRCh38.primary_assembly.genome.fa'
seq_parser = utils.SequenceParser(fasta_path)

genes = ['ABCA8_chr17_68955392_-', 'NFKBIZ_chr3_101849513_+']
gene_seqs = {}

for gene in genes:
    gene_name, chrom, start, strand = gene.split('_')
    seq = seq_parser.extract_seq_centered(chrom, int(start), strand, model.seq_length)
    gene_seqs[gene_name] = seq

In [ ]:
# TSS bin indeces
bins = [447, 448]

In [ ]:
abca8_wt = model.predict(gene_seqs['ABCA8'])[0,:,0]
nfkbiz_wt = model.predict(gene_seqs['NFKBIZ'])[0,:,0]

In [ ]:
utils.plot_track([abca8_wt], color='green', zoom=[0, 896], marks=bins)
utils.plot_track([nfkbiz_wt], color='red', zoom=[0, 896], marks=bins)

# Context swap test
In this example we will swap the TSS tiles (5Kb) of an enhancing context sequence with a silencing one and vice versa. 

To run context swap test we need:
- a loaded model
- onehot encoded sequence (WT) of the source from which the TSS is taken
- onehot encoded sequence (WT) of the target into which the TSS is embedded
- a coordinate interval where the TSS is

In [ ]:
seq_halflen = model.seq_length // 2
half_window_size = 2500
N_shuffles = 10

## TSS (from an enhancing sequence) embedded in a silencing context


In [ ]:
pred_mut = creme.context_swap_test(model, gene_seqs['ABCA8'], gene_seqs['NFKBIZ'],
                                   [seq_halflen - half_window_size, seq_halflen + half_window_size])

In [ ]:
ax=utils.plot_track([abca8_wt], color='green', zoom=[400, 500], alpha=1, label='WT')
utils.plot_track(pred_mut[:,:,0], color='purple', zoom=[400, 500], ax=ax, alpha=1, label='WT TSS + \ncontext from \nsilencing sequence')
plt.legend()


## TSS (from a silencing sequence) embedded in an enhancing context


In [ ]:
pred_mut = creme.context_swap_test(model, gene_seqs['NFKBIZ'], gene_seqs['ABCA8'], 
                                   [seq_halflen - half_window_size, seq_halflen + half_window_size])

In [ ]:
ax=utils.plot_track([nfkbiz_wt], color='red', zoom=[400, 500], alpha=1, label='WT')
utils.plot_track(pred_mut[:,:,0], color='purple', zoom=[400, 500], ax=ax, alpha=1, label='WT TSS + \ncontext from \nenhancing sequence')
plt.legend()